In [1]:
import sys
import os

PROJECT_ROOT = "/home/tom/Documents/UNT/csce6260/projects/OpenPCDet"
sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets.kitti.kitti_dataset import KittiDataset

In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
cfg_file = os.path.join(PROJECT_ROOT, "tools/cfgs/kitti_models/pointrcnn.yaml")
cfg_from_yaml_file(cfg_file, cfg)

dataset = KittiDataset(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    training=False
)

print("Dataset length:", len(dataset))

Dataset length: 3769


In [4]:
sample = dataset[0]
points = sample['points']
print("Points shape:", points.shape)
print("Frame ID:", sample['frame_id'])
print("Sample keys:", sample.keys())

Points shape: (16384, 4)
Frame ID: 000001
Sample keys: dict_keys(['frame_id', 'calib', 'gt_boxes', 'points', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])


In [5]:
points = sample['points'][:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

z = points[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
colors = plt.cm.jet(z_norm)[:, :3]
pcd.colors = o3d.utility.Vector3dVector(colors)

gt_boxes = sample['gt_boxes']
boxes = []

class_colors = {
    1: (1, 0, 0),
    2: (0, 1, 0),
    3: (0, 0, 1),
}

for box in gt_boxes:
    x, y, z, dx, dy, dz, heading, class_idx = box
    class_idx = int(class_idx)
    obb = o3d.geometry.OrientedBoundingBox(
        center=[0, 0, 0],
        R=o3d.geometry.get_rotation_matrix_from_xyz([0, 0, heading]),
        extent=[dx, dy, dz]
    )
    obb.translate([x, y, z])
    obb.color = class_colors.get(class_idx, (1, 1, 1))
    boxes.append(obb)

o3d.visualization.draw_geometries([pcd, *boxes])

[Open3D WARNING] GLFW Error: Wayland: The platform does not support setting the window position
[Open3D WARNING] Failed to initialize GLEW.
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
